# Reports {#sec-reports}

::: {.content-hidden when-format="html"}

## Project Setup

Install and load the necessary packages

In [13]:
#| echo: false
#| output: false
import os
os.environ['R_HOME'] = f'C:/Users/{os.environ.get('USERNAME')}/Miniconda3/envs/r_python_jl/Lib/R'

In [14]:
#| echo: false
#| output: false
%load_ext rpy2.ipython
# only have to run once to allow the R magic command

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


::: {.panel-tabset}

#### R

In [15]:
%%capture 
%%R

library("dplyr")
library("jsonlite")
library("tidyr")
library("REDCapR")
library("knitr")
library("remotes")
library("gt")

In [16]:
%%capture --no-display --no-stdout
%%R

version <- packageVersion("REDCapR")
version

[1] '1.1.9005'


In this project, we will use the bleeding edge version of REDCapR available on Github

In [8]:
%%capture --no-display --no-stdout
%%R

# Detach REDCapR if already loaded, and download the latest version
if (version!='1.1.9005') {
    detach("package:REDCapR", unload=TRUE)
    remotes::install_github("OuhscBbmc/REDCapR")
    library("REDCapR")
    print(packageVersion("REDCapR"))
}

else {
    print("REDCapR package up to date")
}

[1] "REDCapR package up to date"


#### Python

In [17]:
import redcap
import json
import pandas as pd

:::

Assign your project URL and Token

::: {.panel-tabset}

#### R

In [18]:
%%R
path = paste0("C:/Users/", Sys.getenv("USERNAME"), '/json_api_data.json')
token <- jsonlite::fromJSON(path)$dev_token$'308'
url <- "https://dev-redcap.doh.wa.gov/api/"

#### Python

In [19]:
path_to_json = f"C:/Users/{os.environ.get('USERNAME')}/json_api_data.json"
api_key = json.load(open(path_to_json))
api_token = api_key['dev_token']['308']
api_url = api_key['dev_url']
project = redcap.Project(api_url, api_token)

:::

:::

To export reports via API you will need the 'Report ID' of your desired report. You can find the 'Report ID' of the report you want to export by going into your REDCap project and selecting 'Data Export, Reports, and Stats', then 'My Reports & Exports'.  

![](./files/REDCAp_project.png){width=700, fig-align="center"}

## Exporting Raw Reports

In this example, we will export the 'Cases in October 2023' report as seen in the image above. 

::: {.panel-tabset}

#### R

`redcap_report()`

In [12]:
%%capture
%%R
cases_oct_2023 <- redcap_report(
    redcap_uri = url, 
    token = token, 
    report_id = 2178
)$data

::: {.content-hidden when-format="html"}

In [13]:
%%R
tbl<- gt(head(cases_oct_2023))
gt::gtsave(tbl, filename = 'export_reports1.html',  path = "./files/")

:::

<p align="center"><iframe width="100%" height="400" src="./files/export_reports1.html" title="Quarto Documentation"></iframe></p>

#### Python

`export_report()`

In [20]:
project.export_report(report_id='2178', format_type='df', raw_or_label='raw').reset_index().head(7)

,record_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,first_name,last_name,phone_num,zip_code,dob,age,...,symptoms_yesno,symptom_onset,symptoms_exp,symptom_notes,test_yesno,test_positive_yesno,test_positive_date,test_upload,prior_covid_yesno,prior_covid_date
0,1,notifications_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,personal_info_arm_1,NaN,NaN,John,Doe,(999) 999-9999,98105.0,2006-04-11,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,case_intake_arm_1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2023-10-09,"1,2,7",NaN,1.0,1.0,2023-10-10,test_api_export_image.png,0.0,NaN
3,2,notifications_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,personal_info_arm_1,NaN,NaN,Jane,Doe,(999) 999-9999,98105.0,1994-06-29,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,case_intake_arm_1,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2021-06-05,"1,2,3,4,5,7,8,9",NaN,1.0,1.0,2021-06-07,test_api_export_pdf.pdf,0.0,NaN
6,2,case_intake_arm_1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2023-10-11,"1,2,8",NaN,1.0,1.0,2023-10-12,test_api_export_pdf.pdf,1.0,2021-06-06


:::

## Exporting Labeled Reports {#sec-label_reports}
Reports can also be exported as labeled data.

::: {.panel-tabset}

#### R

In [15]:
%%capture
%%R
cases_oct_2023_labeled <- redcap_report(
    redcap_uri = url, 
    token = token,
    report_id = 2178, 
    raw_or_label = 'label'
)$data

::: {.content-hidden when-format="html"}

In [16]:
%%R
tbl<- gt(head(cases_oct_2023_labeled))
gt::gtsave(tbl, filename = 'export_reports2.html', path = "./files/")

:::

#### Python

In [21]:
project.export_report(report_id='2178', format_type='df', raw_or_label='label').reset_index().head(7)

,record_id,redcap_event_name,redcap_repeat_instrument,redcap_repeat_instance,first_name,last_name,phone_num,zip_code,dob,age,...,symptoms_yesno,symptom_onset,symptoms_exp,symptom_notes,test_yesno,test_positive_yesno,test_positive_date,test_upload,prior_covid_yesno,prior_covid_date
0,1,Notifications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Personal Info,NaN,NaN,John,Doe,(999) 999-9999,98105.0,2006-04-11,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Case Intake,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,2023-10-09,"Sore Throat,Cough,Congestion",NaN,Yes,Yes,2023-10-10,test_api_export_image.png,No,NaN
3,2,Notifications,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,Personal Info,NaN,NaN,Jane,Doe,(999) 999-9999,98105.0,1994-06-29,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,Case Intake,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,2021-06-05,"Sore Throat,Cough,Shortness of Breath,Chest Pa...",NaN,Yes,Yes,2021-06-07,test_api_export_pdf.pdf,No,NaN
6,2,Case Intake,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,Yes,2023-10-11,"Sore Throat,Cough,Fever",NaN,Yes,Yes,2023-10-12,test_api_export_pdf.pdf,Yes,2021-06-06


**Note:** Exporting labeled headers only works when `format_type='csv'`. See @sec-csv for more information. 

:::